<a href="https://colab.research.google.com/github/misrori/pyladies/blob/main/ingatlanok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install scrapethat
from bs4 import BeautifulSoup
import requests
import pandas as pd
import cloudscraper

def read_html(link):
    response = requests.get(link)
    return(BeautifulSoup(response.text, 'html'))

def read_cloud(link):
    scraper = cloudscraper.create_scraper()  # returns a CloudScraper instance
    response = scraper.get(link)
    return(BeautifulSoup(response.text, 'html'))

def get_texts(link_nodes):
    return([x.text for x in link_nodes])

def strip_texts(my_texts):
    return([x.strip() for x in my_texts])

def get_links(link_nodes):
    return([x['href'] for x in link_nodes])

def paste0(base_link, numbers):
    return([f'{base_link}{s}' for s in numbers])

def fromjson(link):
    response = requests.get(link)
    return(response.json())

In [ ]:
t = read_html('https://www.ingatlannet.hu/elado-lakas')

In [ ]:
t.select('#searchListItems')[0].select('.col-md-8')[0].find('a')['href']

'/elado-lakas-xi-kerulet/6532552'

In [ ]:
a_list = t.select('#searchListItems')[0].select('.col-md-8')
links = [f"https://www.ingatlannet.hu{x.find('a')['href']}"  for x in a_list]

In [ ]:
t = read_html('https://www.ingatlannet.hu/elado-lakas-szazhalombatta/6559806')

In [ ]:
keys = strip_texts(get_texts(t.select('dt')))
keys

['Közös költség',
 'Emelet',
 'Szobák száma',
 'Fűtés',
 'Lift',
 'Konyha',
 'Fürdőszoba',
 'Erkély']

In [ ]:
values = strip_texts(get_texts(t.select('dd')))
values

['nincs megadva',
 '8.',
 '2',
 'gáz (központi)',
 'van',
 'nincs megadva',
 'nincs megadva',
 'nincs megadva']

In [ ]:
data = dict(zip(keys, values))

In [ ]:
data['description'] = t.select('.description-text')[0].text.strip()

'Az ingatlanról:\nSzázhalombatta kedvelt részén eladó, egy panorámás, erkélyes, 2 szobás, 8. emeleti panel lakás.A szobák külön bejáratúak, de egybe is nyithatók. Az épület hőszigetelése megtörtént, a lépcsőház rendezett, lift van!Fűtés költség 10.000 FtInfrastruktúra, közlekedés jó!Irányára:31.500.000 Ft\nA környékről:\nParkra néző, panorámás, jól megközelíthető helyen lévő társasház.'

In [ ]:
def get_one_page(link):
    t = read_html(link)
    keys = strip_texts(get_texts(t.select('dt')))
    values = strip_texts(get_texts(t.select('dd')))
    data = dict(zip(keys, values))
    data['description'] = t.select('.description-text')[0].text.strip()
    return(data)

In [ ]:
df = pd.DataFrame(list(map(get_one_page, links[1:5])))

In [ ]:
df

,Közös költség,Emelet,Szobák száma,Ingatlan állapota,Fűtés,Lift,description,Építés éve,Konyha,Kamra,Fürdőszoba,Wc,étkező,Erkély,Pince,Hall
0,nincs megadva,1.,3 (1 + 2),felújított,konvektor,nincs,Az ingatlanról:\nSzolnok belvárosában a Szántó...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nincs megadva,3.,2,NaN,távfűtés,van,Az ingatlanról:\nCsak Irodánk kínálatában!Elad...,1980,nincs megadva,nincs megadva,nincs megadva,NaN,NaN,NaN,NaN,NaN
2,nincs megadva,3.,3 (1 + 2),NaN,gáz (egyedi),nincs,"Az ingatlanról:\nEladó, Budapest, 18. kerület ...",2003,nincs megadva,NaN,nincs megadva,nincs megadva,nincs megadva,nincs megadva,NaN,NaN
3,nincs megadva,3.,1,NaN,egyéb,nincs,Az ingatlanról:\nKiváló Befektetési Lehetőség!...,1920,nincs megadva,nincs megadva,nincs megadva,nincs megadva,NaN,NaN,nincs megadva,nincs megadva


# ingatlanok.hu


In [ ]:
from scrapethat import *

In [ ]:
t = read_html('https://ingatlanok.hu/elado/lakas?page=2')


In [ ]:
t.select('.feat_property')[0].select('a')[0]['href']

'https://ingatlanok.hu/elado/lakas/budapest-xxi-ker/19-millio-ft;47-negyzetmeter;2-szoba;tegla-epitesu/11258136'

In [ ]:
t.find_all(class_='feat_property')[0].find_all('a')[0]['href']

'https://ingatlanok.hu/elado/lakas/budapest-xxi-ker/19-millio-ft;47-negyzetmeter;2-szoba;tegla-epitesu/11258136'

In [ ]:
boxes = t.select('.feat_property')

In [ ]:
house_links = [x.find_all('a')[0]['href'] for x in boxes]
house_links

links = [x for x in house_links if x.startswith('http')]

# Egy oldal letöltése


In [ ]:
link= 'https://ingatlanok.hu/elado/lakas/budapest-xxi-ker/19-millio-ft;47-negyzetmeter;2-szoba;tegla-epitesu/11258136'
t = read_html(link)

In [ ]:
keys = get_texts(t.select('.left'))
values = get_texts(t.select('.col-xl-6 span'))

In [ ]:
dict(zip(keys, values))

{'Ár:': '18 700 000 Ft',
 'Méret:': '47 négyzetméter',
 'Azonosító:': '11258136',
 'Irodai azonosító:': 'LK042352-4426683',
 'Emelet:': '2. emelet',
 'Építőanyag:': 'Tégla',
 'Fűtés jellege:': 'Villany',
 'Ingatlan állapota:': 'Jó',
 'Építési év:': '21-50 éves',
 'Egész szobák:': '2'}

In [ ]:
def get_one_page(link):
    t = read_html(link)
    keys = get_texts(t.select('.left'))
    values = get_texts(t.select('.col-xl-6 span'))
    data = dict(zip(keys, values))
    return(data)


In [ ]:
import pandas as pd
df = pd.DataFrame(list(map(get_one_page, links )))

In [ ]:
df

# ingatlanbazar


In [ ]:
from scrapethat import *

In [ ]:
t = read_html('https://www.ingatlanbazar.hu/ingatlan-Elado-Lakas-es-Haz?property__2=3_2&page=2')

# Json

In [ ]:
scripts = t.find_all('script')

In [ ]:
textem = re.search(r'var jsonData = (.*);', scripts[0].string)
data = json.loads(textem.group(1))
data['property-search']

In [ ]:
import json
with open('data.json', 'w') as f:
    json.dump(data['property-search'], f)

In [ ]:
import re
import json

# Find the script element containing jsonData
script_element = t.find('script', text=re.compile(r'var jsonData = .*;'))

if script_element:
    # Extract the JSON data using regex
    match = re.search(r'var jsonData = (.*);', script_element.string)

    if match:
        json_data = json.loads(match.group(1))
        print(json_data)
    else:
        print("JSON data not found in the script element.")
else:
    print("Script element with JSON data not found.")

{'property-search': {'oldParams': {'property__2': '3_2', 'page': 2, 'property_location': []}, 'queryString': 'property__2=3_2&page=2', 'hits': [{'propertyId': '43d35a46-5fb5-11ee-9d08-005056b3506b', 'oldId': None, 'status': 1, 'ownProperty': 1, 'reportCount': '0', 'originCreatedAt': '2023-09-30 19:17:42', 'createdAt': '2023-09-30 19:17:42', 'startDate': '2023-09-30', 'endDate': '2023-12-30', 'priceHUF': 26500000, 'sqmPriceHuf': 500000, 'priceEUR': 69667, 'sqmPriceEur': 1314.47, 'currencyOriginal': 'HUF', 'title': 'Uránváros, Pécs, ingatlan, eladó, lakás, 53 m2', 'userTitle': 'Egyetemhez közel erkélyes, karbantartott lakás eladó.', 'description': 'Uránváros, Pollack Mihály utcában távfűtéses, összkomfortos, erkélyes, karbantartott jó állapotú lakás eladó.Saját tárolóval rendelkezik, azonnal költözhető és az ár alkuképes.Tel.:+36306127092', 'ownerId': '489377', 'salesPersonId': None, 'projectId': None, 'agencyInternalId': None, 'county': 'Baranya megye', 'subregion': 'Pécsi kistérség', '

<ipython-input-36-b296c841e8cc>:5: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_element = t.find('script', text=re.compile(r'var jsonData = .*;'))
